In [5]:
# from kwnlp_dump_downloader import download_jobs
# wp_yyyymmdd = "20230820"
# wd_yyyymmdd = "20230821"
# data_path = "/Users/tabdull1/Python_Projects/Wiki_dumps/"
# jobs_to_download = ["categorytable", "articlesdump","categorylinkstable"]
# wiki = "enwiki"
# download_jobs(wp_yyyymmdd, wd_yyyymmdd, data_path=data_path, jobs_to_download=jobs_to_download, wiki=wiki)

In [2]:
# import pandas as pd

# # Read the text file
# with open('/Users/tabdull1/Python_Projects/data/text/enwiki-20230820-pages-articles-multistream.xml-0001.txt', 'r', encoding='utf-8') as f:
#     lines = f.readlines()

# # Initialize variables to store data
# data = []
# title = ""
# categories = ""
# content = ""
# is_in_content = False

# # Iterate through the lines and extract data
# for line in lines:
#     line = line.strip()
#     if line.startswith("[[") and line.endswith("]]"):
#         # Found a title
#         if title:
#             data.append({'title': title, 'categories': categories, 'content': content})
#             title = ""
#             categories = ""
#             content = ""
#         title = line[2:-2]
#     elif line.startswith("CATEGORIES:"):
#         # Found categories
#         categories = line.split(":")[1].strip()
#     elif line.startswith("==") and line.endswith("=="):
#         # Found a section heading
#         is_in_content = not is_in_content
#     elif not is_in_content:
#         # Accumulate content
#         content += line + " "
#     elif is_in_content:
#         # Accumulate content
#         content += line + " "

# # Append the last entry
# data.append({'title': title, 'categories': categories, 'content': content})

# # Create a DataFrame
# df = pd.DataFrame(data)



In [3]:
import os
import pandas as pd

# Path to the folder containing text files
folder_path = '/Users/tabdull1/Python_Projects/data/text'


# Initialize a list to store extracted data
data = []

# Iterate through the files in the folder

for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # Initialize variables to store data
        title = ""
        categories = ""
        content = ""
        is_in_content = False
        
        # Iterate through the lines and extract data
        for line in lines:
            line = line.strip()
            if line.startswith("[[") and line.endswith("]]"):
                if title:
                    data.append({'title': title, 'categories': categories, 'content': content})
                    title = ""
                    categories = ""
                    content = ""
                title = line[2:-2]
            elif line.startswith("CATEGORIES:"):
                categories = line.split(":")[1].strip()
            elif line.startswith("==") and line.endswith("=="):
                is_in_content = not is_in_content
            elif not is_in_content:
                content += line + " "
            elif is_in_content:
                content += line + " "
        
        # Append the last entry
        data.append({'title': title, 'categories': categories, 'content': content})



In [4]:
# Create a DataFrame
df = pd.DataFrame(data)


In [5]:
df

,title,categories,content
0,Anarchism,"Anarchism, Anti-capitalism, Anti-fascism, Econ...",Anarchism is a political philosophy and move...
1,Autism,1910s neologisms,
2,Albedo,"Land surface effects on climate, Climate chang...",Albedo (æ; la) is the fraction of sunlight t...
3,A,"ISO basic Latin letters, Vowel letters","A, or a, is the first letter and the first v..."
4,Alabama,"Alabama, 1819 establishments in the United Sta...","Alabama (,) is a state in the Southeastern r..."
...,...,...,...
7087731,Texas's 32nd House of Representatives district,Texas House of Representatives districts,District 32 is a district in the Texas House...
7087732,Michael Daniel,,Michael Daniel may refer to:
7087733,Artifact (app),"Android (operating system) software, IOS softw...",Artifact is a personalized news app that mak...
7087734,George Fortescue (MP),"1791 births, 1877 deaths, Younger sons of earl...",George Matthew Fortescue (21 May 1791 – 24 J...


In [6]:
# Preprocess the DataFrame
def preprocess_dataframe(df):
    # Remove duplicates
    df = df.drop_duplicates(subset=['title', 'categories', 'content'])
    
    # Remove rows with empty categories
    df = df[df['categories'].apply(len) > 0]
    
    # Remove rows with sentence length less than 20
    df = df[df['content'].apply(lambda x: len(x.split()) >= 20)]
    df =df.reset_index(drop=True)
    
    return df

# folder_path = 'path_to_your_folder'  # Replace with the actual path to your folder
# df = load_data_from_folder(folder_path)
preprocessed_df = preprocess_dataframe(df)


In [7]:
preprocessed_df

,title,categories,content
0,Anarchism,"Anarchism, Anti-capitalism, Anti-fascism, Econ...",Anarchism is a political philosophy and move...
1,Albedo,"Land surface effects on climate, Climate chang...",Albedo (æ; la) is the fraction of sunlight t...
2,A,"ISO basic Latin letters, Vowel letters","A, or a, is the first letter and the first v..."
3,Alabama,"Alabama, 1819 establishments in the United Sta...","Alabama (,) is a state in the Southeastern r..."
4,Achilles,"Greek mythological heroes, Kings of the Myrmid...","In Greek mythology, Achilles (ə ə) or Achill..."
...,...,...,...
5848885,Atrå,"Tinn, Villages in Vestfold og Telemark",Atrå is a village in Tinn Municipality in Ve...
5848886,Texas's 32nd House of Representatives district,Texas House of Representatives districts,District 32 is a district in the Texas House...
5848887,Artifact (app),"Android (operating system) software, IOS softw...",Artifact is a personalized news app that mak...
5848888,George Fortescue (MP),"1791 births, 1877 deaths, Younger sons of earl...",George Matthew Fortescue (21 May 1791 – 24 J...


In [14]:
preprocessed_df[preprocessed_df['title'] == 'Christianity'].categories.iloc[0]

'Christianity, 1st-century establishments, 1st-century introductions, Abrahamic religions, Monotheistic religions, Western culture'

In [15]:
#select rows with categories containing "Machine learning"
# preprocessed_df[preprocessed_df['categories'].apply(lambda x: 'government' in x)]

In [16]:
#create doc ids for each document in the dataframe
list = []

for i in range(1, preprocessed_df.shape[0] + 1): # gets the length of the DataFrame.
    list.append(f'doc{i:02d}') # Using f-string for format and 02d for leading zeros.

preprocessed_df['docno'] = list

preprocessed_df.head()

,title,categories,content,docno
0,Anarchism,"Anarchism, Anti-capitalism, Anti-fascism, Econ...",Anarchism is a political philosophy and move...,doc01
1,Albedo,"Land surface effects on climate, Climate chang...",Albedo (æ; la) is the fraction of sunlight t...,doc02
2,A,"ISO basic Latin letters, Vowel letters","A, or a, is the first letter and the first v...",doc03
3,Alabama,"Alabama, 1819 establishments in the United Sta...","Alabama (,) is a state in the Southeastern r...",doc04
4,Achilles,"Greek mythological heroes, Kings of the Myrmid...","In Greek mythology, Achilles (ə ə) or Achill...",doc05


In [17]:

preprocessed_df.columns = ['title',"categories","text","docno"]


In [18]:
preprocessed_df

,title,categories,text,docno
0,Anarchism,"Anarchism, Anti-capitalism, Anti-fascism, Econ...",Anarchism is a political philosophy and move...,doc01
1,Albedo,"Land surface effects on climate, Climate chang...",Albedo (æ; la) is the fraction of sunlight t...,doc02
2,A,"ISO basic Latin letters, Vowel letters","A, or a, is the first letter and the first v...",doc03
3,Alabama,"Alabama, 1819 establishments in the United Sta...","Alabama (,) is a state in the Southeastern r...",doc04
4,Achilles,"Greek mythological heroes, Kings of the Myrmid...","In Greek mythology, Achilles (ə ə) or Achill...",doc05
...,...,...,...,...
5848885,Atrå,"Tinn, Villages in Vestfold og Telemark",Atrå is a village in Tinn Municipality in Ve...,doc5848886
5848886,Texas's 32nd House of Representatives district,Texas House of Representatives districts,District 32 is a district in the Texas House...,doc5848887
5848887,Artifact (app),"Android (operating system) software, IOS softw...",Artifact is a personalized news app that mak...,doc5848888
5848888,George Fortescue (MP),"1791 births, 1877 deaths, Younger sons of earl...",George Matthew Fortescue (21 May 1791 – 24 J...,doc5848889


In [20]:

# Convert the string representations to actual lists
preprocessed_df['categories'] = preprocessed_df['categories'].apply(lambda x: [x])

In [21]:
preprocessed_df.to_csv("/Users/tabdull1/Python_Projects/data/preprocessed_wiki_dumps.csv", index=False)

In [1]:
#start pyterrier
import pyterrier as pt
import pandas as pd
import pandas as pd
import nltk
import string

if not pt.started():
    pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [23]:
print("Creating index")
indexer = pt.IterDictIndexer("./Big_wiki_index", verbose=True, meta={'docno':20, "title":256, "categories":4096})
index_ref=indexer.index(preprocessed_df.fillna("").to_dict(orient='records'), fields=['title','text' ])
print("Done creating index")

Creating index
Done creating index


In [2]:
# index = pt.IndexFactory.of(index_ref)
index = pt.IndexFactory.of("./Big_wiki_index")

In [3]:
print(index.getCollectionStatistics().toString())

Number of documents: 5848890
Number of terms: 5203906
Number of postings: 805620653
Number of fields: 2
Number of tokens: 1483758157
Field names: [title, text]
Positions:   false



In [4]:
#test on dbpedia
#evaluate
from sklearn.datasets import fetch_20newsgroups
from datasets import load_dataset
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags
from gensim.utils import simple_preprocess

# Download required data for nltk
import re
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


/Users/tabdull1/Python_Projects/RAZL/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 
def tokenize(doc):
        tokenized =  simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

        return ' '.join(tokenized)


# Function to remove stopwords and punctuations from text
def remove_stopwords_punctuations(text):
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and punctuations
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words and word.lower() not in string.punctuation]
    
    # Reconstruct the text from filtered words
    return ' '.join(filtered_words)

def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)



In [8]:
import pandas as pd
# Load the test dataset
# test = load_dataset('dbpedia_14', split='test')
# test = pd.DataFrame(test)

# Convert the dataset to a pandas DataFrame
test = pd.read_csv("/Users/tabdull1/Python_Projects/UMLS_Data/oshumed_test2.csv")

# test['category_name'] = test['label'].replace({0:"World", 1:"Sports", 2:"Business", 3:"Sci/Tech"})
# test['category_name'] = test['label'].map({0:"Companies", 1: "Educational institutions", 2: "Artists", 3: "Athletes", 4: "Politics|Government", 5: "Transport", 
# 6: "Buildings and structures", 7: "Landforms", 8: "Villages", 9: "Animals", 10: "Plants", 11: "Albums", 12: "Films", 13: "Written communication"})

test

,text,label,category_name
0,Hemoptysis. Indications for bronchoscopy.\n In...,8,Respiratory Tract Diseases
1,Spirometric findings and mortality in never-sm...,8,Respiratory Tract Diseases
2,Perioperative effect of methylprednisolone giv...,8,Respiratory Tract Diseases
3,Selective differences in macrophage population...,8,Respiratory Tract Diseases
4,Treatment of intractable aspiration using a la...,8,Respiratory Tract Diseases
...,...,...,...
4038,Cyclocryotherapy in selected cases of congenit...,11,Eye Diseases
4039,Treatment of exophthalmos.\n Current procedure...,11,Eye Diseases
4040,The potential for mental status changes associ...,11,Eye Diseases
4041,Epidemiologic evidence of a role for the antio...,11,Eye Diseases


In [10]:
#apply tokenize function to the columns
test['tokenized'] = test['text'].apply(tokenize)
test['tokenized'] = test['tokenized'].apply(remove_stopwords_punctuations)
test['tokenized'] = test['tokenized'].apply(remove_special_characters)

# # Calculate word counts and filter rows
# test['word_count'] = test['tokenized'].apply(count_words)
# test = test[test['word_count'] > 1]

test = test.reset_index(drop=True)

In [11]:
query = test[["tokenized"]]
#reset index as a column
query.reset_index(inplace=True)
query.columns= ['qid', 'query']
query.head()

,qid,query
0,0,hemoptysis indications bronchoscopy indication...
1,1,spirometric findings mortality never smokers r...
2,2,perioperative effect given lung surgery plasma...
3,3,selective differences macrophage populations m...
4,4,treatment intractable aspiration using larynge...


In [12]:
# batch retrieve
bm25_news = pt.BatchRetrieve(index, num_results=100, wmodel="BM25", metadata=["docno", 'title', "categories"]).transform(query)


/Users/tabdull1/Python_Projects/RAZL/lib/python3.9/site-packages/pyterrier/batchretrieve.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queries['qid'] = queries['qid'].astype(str)


In [13]:
bm25_news

,qid,docid,docno,title,categories,rank,score,query
0,0,1562400,doc1562401,Bronchoscopy,"[""Endoscopy, Respiratory system procedures, Ve...",0,86.543910,hemoptysis indications bronchoscopy indication...
1,0,2752285,doc2752286,Electromagnetic navigation bronchoscopy,"[""Interventional radiology""]",1,78.165361,hemoptysis indications bronchoscopy indication...
2,0,3456756,doc3456757,Foreign body aspiration,"[""Symptoms and signs""]",2,74.155948,hemoptysis indications bronchoscopy indication...
3,0,4470325,doc4470326,Tracheoinnominate fistula,"[""Fistulas""]",3,73.286398,hemoptysis indications bronchoscopy indication...
4,0,104618,doc104619,Hemoptysis,"[""Bleeding, Respiratory diseases, Symptoms and...",4,70.064945,hemoptysis indications bronchoscopy indication...
...,...,...,...,...,...,...,...,...
404295,4042,97778,doc97779,Antisocial personality disorder,"[""Anti-social behaviour, Behavioural sciences,...",95,47.597752,factors associated delay diagnosis childhood a...
404296,4042,16781,doc16782,Asperger syndrome,"[""Asperger syndrome, Autism, Autism spectrum d...",96,47.567311,factors associated delay diagnosis childhood a...
404297,4042,3436542,doc3436543,Separation anxiety disorder,"[""Anxiety disorders, Attachment theory, Mental...",97,47.485337,factors associated delay diagnosis childhood a...
404298,4042,5706546,doc5706547,Cystic fibrosis and race,"[""Cystic fibrosis, Race and health""]",98,47.450985,factors associated delay diagnosis childhood a...


In [59]:
# bm25_newsX = bm25_news.copy()
# bm25_news = bm25_newsX.copy()
# bm25_news = bm25_news.groupby(['qid']).head(1)

In [14]:

import re

# Identify rows with incomplete patterns using regular expressions
incomplete_pattern_rows =bm25_news[bm25_news['categories'].str.count('\[') != bm25_news['categories'].str.count('\]')]

# Identify rows with incomplete patterns using regular expressions
incomplete_pattern_rows = incomplete_pattern_rows[incomplete_pattern_rows['categories'].str.contains(r'^\[".*[^]]$', regex=True)]


# # Update incomplete patterns using vectorized operations
bm25_news.loc[incomplete_pattern_rows.index, 'categories'] = bm25_news.loc[incomplete_pattern_rows.index, 'categories'].apply(lambda x: x + '"]')



In [15]:
#convert colunm to int
bm25_news['qid'] = bm25_news['qid'].astype(int)
bm25_news.categories.iloc[0]

'["Endoscopy, Respiratory system procedures, Veterinary diagnosis, Articles containing video clips"]'

In [16]:
import pandas as pd
import ast

# Convert the strings to actual lists using ast.literal_eval
bm25_news['categories'] = bm25_news['categories'].apply(ast.literal_eval)

# Group by 'id' and aggregate rows with type '["Business"]' into a single list
news_grouped = bm25_news.groupby('qid', as_index=False)['categories'].agg(sum)



In [17]:
len(news_grouped)

4043

In [19]:
# test

In [120]:
# news_grouped.iloc[69999].categories#.count("Business")

In [121]:
# # Specify the file path
# file_path = "test2.txt"

# # Open the file for writing
# with open(file_path, "w") as file:
#     # Write each element of the list to a separate line
#     for item in news_grouped.iloc[0].categories:
#         file.write(item + "\n")

# # print("List saved to", file_path)

In [20]:
ids = news_grouped.qid.values.tolist()
full_range = set(range(1, len(query)))
missing_ids = [num for num in full_range if num not in ids]
missing_ids

[]

In [21]:
#convert colunm to int
query['qid'] = query['qid'].astype(int)
test

/var/folders/nf/xq6nf21s6vq1j7z3gbrc5ftr0000gn/T/ipykernel_3944/2700284474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['qid'] = query['qid'].astype(int)


,text,label,category_name,tokenized
0,Hemoptysis. Indications for bronchoscopy.\n In...,8,Respiratory Tract Diseases,hemoptysis indications bronchoscopy indication...
1,Spirometric findings and mortality in never-sm...,8,Respiratory Tract Diseases,spirometric findings mortality never smokers r...
2,Perioperative effect of methylprednisolone giv...,8,Respiratory Tract Diseases,perioperative effect given lung surgery plasma...
3,Selective differences in macrophage population...,8,Respiratory Tract Diseases,selective differences macrophage populations m...
4,Treatment of intractable aspiration using a la...,8,Respiratory Tract Diseases,treatment intractable aspiration using larynge...
...,...,...,...,...
4038,Cyclocryotherapy in selected cases of congenit...,11,Eye Diseases,selected cases congenital glaucoma investigate...
4039,Treatment of exophthalmos.\n Current procedure...,11,Eye Diseases,treatment exophthalmos current procedures grav...
4040,The potential for mental status changes associ...,11,Eye Diseases,potential mental status changes associated sys...
4041,Epidemiologic evidence of a role for the antio...,11,Eye Diseases,epidemiologic evidence role antioxidant vitami...


In [22]:
filtered_df = query.copy()
filtered_df["Truth"]=test['category_name']
filtered_df["Label"]=test['label']

In [23]:
# Filter out rows where 'column_name' values are present in 'items_to_remove' list
filtered_df = filtered_df[~filtered_df['qid'].isin(missing_ids)]
#reset index of filtered_df
filtered_df = filtered_df.reset_index(drop=True)

In [24]:
# Merge both DataFrames on 'qid' while maintaining the ordering
merged_df = pd.merge_ordered(news_grouped, filtered_df, on='qid',how='inner')

In [25]:
merged_df

,qid,categories,query,Truth,Label
0,0,"[Endoscopy, Respiratory system procedures, Vet...",hemoptysis indications bronchoscopy indication...,Respiratory Tract Diseases,8
1,1,"[Pulmonary function testing, Respiratory physi...",spirometric findings mortality never smokers r...,Respiratory Tract Diseases,8
2,2,"[Blood, Transfusion medicine, Surgical procedu...",perioperative effect given lung surgery plasma...,Respiratory Tract Diseases,8
3,3,"[Anatomical pathology, Inflammations, Macropha...",selective differences macrophage populations m...,Respiratory Tract Diseases,8
4,4,"[Cardiac procedures, Vascular surgery, Neurosu...",treatment intractable aspiration using larynge...,Respiratory Tract Diseases,8
...,...,...,...,...,...
4038,4038,"[Optic nerve, Eye surgery, Glaucoma, Minimally...",selected cases congenital glaucoma investigate...,Eye Diseases,11
4039,4039,"[Dog diseases, Disorders of eyelid, lacrimal s...",treatment exophthalmos current procedures grav...,Eye Diseases,11
4040,4040,"[Cognitive disorders, Health psychology, Geria...",potential mental status changes associated sys...,Eye Diseases,11
4041,4041,"[Carotenoids, Vitamin A, Hydrocarbons, Tetrate...",epidemiologic evidence role antioxidant vitami...,Eye Diseases,11


In [26]:
def list_to_sentence(string_list):
    return " ".join(string_list)

# Apply the function to the DataFrame column
merged_df['sentence'] = merged_df['categories'].apply(list_to_sentence)

In [27]:
merged_df

,qid,categories,query,Truth,Label,sentence
0,0,"[Endoscopy, Respiratory system procedures, Vet...",hemoptysis indications bronchoscopy indication...,Respiratory Tract Diseases,8,"Endoscopy, Respiratory system procedures, Vete..."
1,1,"[Pulmonary function testing, Respiratory physi...",spirometric findings mortality never smokers r...,Respiratory Tract Diseases,8,"Pulmonary function testing, Respiratory physio..."
2,2,"[Blood, Transfusion medicine, Surgical procedu...",perioperative effect given lung surgery plasma...,Respiratory Tract Diseases,8,"Blood, Transfusion medicine, Surgical procedur..."
3,3,"[Anatomical pathology, Inflammations, Macropha...",selective differences macrophage populations m...,Respiratory Tract Diseases,8,"Anatomical pathology, Inflammations Macrophage..."
4,4,"[Cardiac procedures, Vascular surgery, Neurosu...",treatment intractable aspiration using larynge...,Respiratory Tract Diseases,8,"Cardiac procedures Vascular surgery, Neurosurg..."
...,...,...,...,...,...,...
4038,4038,"[Optic nerve, Eye surgery, Glaucoma, Minimally...",selected cases congenital glaucoma investigate...,Eye Diseases,11,"Optic nerve Eye surgery, Glaucoma, Minimally i..."
4039,4039,"[Dog diseases, Disorders of eyelid, lacrimal s...",treatment exophthalmos current procedures grav...,Eye Diseases,11,"Dog diseases, Disorders of eyelid, lacrimal sy..."
4040,4040,"[Cognitive disorders, Health psychology, Geria...",potential mental status changes associated sys...,Eye Diseases,11,Cognitive disorders Health psychology Geriatri...
4041,4041,"[Carotenoids, Vitamin A, Hydrocarbons, Tetrate...",epidemiologic evidence role antioxidant vitami...,Eye Diseases,11,"Carotenoids, Vitamin A, Hydrocarbons, Tetrater..."


In [28]:
# merged_df.to_csv("/Users/tabdull1/Python_Projects/data/merged_df_BM25.csv", index=False)
merged_df.to_csv("/Users/tabdull1/Python_Projects/data/oshumed_merged_df.csv", index=False)

In [133]:
# merged_df

,qid,categories,query,Truth,Label,sentence
0,0,"[Sake, Beer in Vietnam, Manufacturing companie...",ty ku taku american alcoholic beverage company...,Companies,0,"Sake Beer in Vietnam, Manufacturing companies ..."
1,1,"[2001 establishments in California, 2015 dises...",oddlot entertainment founded longtime producer...,Companies,0,"2001 establishments in California, 2015 disest..."
2,2,"[Henkel, Henkel brands, Chemical companies of ...",henkel ag company kgaa operates worldwide lead...,Companies,0,"Henkel, Henkel brands, Chemical companies of G..."
3,3,"[Companies based in Sydney, Privately held com...",goat store games type store llc one largest re...,Companies,0,"Companies based in Sydney, Privately held comp..."
4,4,[Defunct aircraft manufacturers of the United ...,ragwing aircraft designs also called ragwing a...,Companies,0,Defunct aircraft manufacturers of the United S...
...,...,...,...,...,...,...
69995,69995,"[2007 non-fiction books, 2007 in the environme...",energy victory winning war terror breaking fre...,Written communication,13,"2007 non-fiction books, 2007 in the environmen..."
69996,69996,"[1951 short story collections, Postmodern book...",bestiario book short stories written julio cor...,Written communication,13,"1951 short story collections, Postmodern books..."
69997,69997,"[1816 births, 1855 deaths, 19th-century Englis...",wuthering heights novel emily bronte written o...,Written communication,13,"1816 births, 1855 deaths, 19th-century English..."
69998,69998,"[1988 establishments in Luxembourg, Newspapers...",independant newspaper published luxembourg,Written communication,13,"1988 establishments in Luxembourg, Newspapers ..."


In [7]:
import os
import pandas as pd

folder_path = '/Users/tabdull1/Python_Projects/data/text'  # Replace with the actual path to your folder
file_order = sorted(os.listdir(folder_path))  # Get the list of filenames in alphabetical order

data_list = []

for filename in file_order:
    file_path = os.path.join(folder_path, filename)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
        title = lines[0].strip('[]\n')
        categories_line = lines[2].strip('CATEGORIES: ').split(', ')
        categories = categories_line[0].split(', ')
        content = ' '.join(lines[4:]).replace('\n', ' ').replace('==', '')  # Remove section headings
        
        data_list.append({'title': title, 'categories': categories, 'content': content})

df = pd.DataFrame(data_list)

# # Sort the DataFrame by 'title' column
# df_sorted = df.sort_values(by='title')

# # Display the sorted DataFrame
# print(df_sorted)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 22: invalid continuation byte